In [60]:
import requests 
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
#import time
#import random as rand 

import pandas as pd
from google.colab import files


#Game URL

In [49]:
game = 'gran-turismo-7'
platform = 'playstation-5'

In [50]:
url = f'https://www.metacritic.com/game/{platform}/{game}/user-reviews'

In [51]:
# url = 'https://www.metacritic.com/game/switch/pokemon-sword/user-reviews?page='+str(page)
# url = 'https://www.metacritic.com/game/playstation-5/elden-rin/user-reviews?page='+str(page)
# url = 'https://www.metacritic.com/game/playstation-5/gran-turismo-7/user-reviews?page=0'
# url = 'https://www.metacritic.com/game/playstation-5/ghostwire-tokyo/user-reviews?page=0'

#Parse

In [52]:
user_agent = {'User-agent': 'Mozilla/5.0'}
response  = requests.get(f'{url}?page=0', headers = user_agent)
#time.sleep(rand.randint(3,30)) 
soup = BeautifulSoup(response.text, 'html.parser')
lastPage = soup.find('li',class_='page last_page')
print(lastPage.text.strip("…"))
pageInt = int(lastPage.text.strip("…"))

69


In [59]:
def is_other_language(texts):
    # Korean
    if re.search("[\uac00-\ud7a3]", texts):
      return True
    # Japanese
    if re.search("[\u3040-\u30ff]", texts):
      return True
    # Chinese
    if re.search("[\u4e00-\u9FFF]", texts):
      return True
    # Cyrillic 
    if re.search('[\u0400-\u04FF]', texts):
      return True

    return False

In [66]:
import string
printable = set(string.printable)

def clean_web(text):
  text = text.replace('\r', ' ')
  text = text.replace('\r\n', ' ')
  text = text.replace('\n', ' ')
  text = text.replace('’', '\'')
  text = text.replace('‘', '\'')
  return ''.join(filter(lambda x: x in printable, text))

In [67]:
review_dict = {
  'name':[],
  'date':[],
  'rating':[],
  'review':[]
}

for page in tqdm(range(pageInt)): #Remember to update the number of pages
  user_agent = {'User-agent': 'Mozilla/5.0'}
  response = requests.get(f'{url}?page={page}', headers=user_agent)
  soup = BeautifulSoup(response.text, 'html.parser')
  for review in soup.find_all('div', class_='review_content'):
    if review.find('div', class_='name') == None:
      break

    added_review = False
    if review.find('span', class_='blurb blurb_expanded'):
      found_review = review.find('span', class_='blurb blurb_expanded')
      if found_review and not is_other_language(found_review.text):
        added_review = True
        review_dict['review'].append(clean_web(found_review.text))
    else:
      found_review = review.find('div', class_='review_body').find('span')
      if found_review and not is_other_language(found_review.text):
        added_review = True
        review_dict['review'].append(clean_web(found_review.text))

    if added_review:
      review_dict['name'].append(review.find('div', class_='name').find('a').text.strip())
      review_dict['date'].append(review.find('div', class_='date').text.strip())
      review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text.strip())


100%|██████████| 69/69 [00:30<00:00,  2.25it/s]


#Create DataFrame and Print It

In [68]:
reviews = pd.DataFrame(review_dict)
reviews

,name,date,rating,review
0,Jrf887,"Apr 9, 2022",0,This is gaming history on the making cause EU...
1,acid_,"Mar 18, 2022",0,Few days ago I was really happy with this game...
2,Whasze,"Mar 17, 2022",0,Screw this game. I have been a GT fan since I ...
3,baqfish,"Mar 17, 2022",0,Could have been so good but the game is pretty...
4,Checkman86,"Mar 17, 2022",0,first of all: this game should be F2P with thi...
...,...,...,...,...
6733,Jokeyy,"Mar 26, 2022",0,Omessa la fiat seicento(miglior auto da corsa ...
6734,Alikzam,"Mar 26, 2022",0,"Premium price tag, freemium feel. You pay 70 f..."
6735,CaptnYellowBear,"Mar 29, 2022",0,As a person who has to move to a place where I...
6736,FFe1Cv,"Apr 20, 2022",0,Sony is isnulting the gamers. How? Heres how s...


#Send to CSV

In [69]:
reviews.to_csv('metacritic.csv', encoding='utf-8')
files.download('metacritic.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>